In [1]:
import pandas as pd
import numpy as np

# Additional modules
import sys

# Data visualisation 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dataframe = pd.read_excel('english_scored_body_only.xlsx')

In [3]:
dataframe.head()

,SNo,date,heading,body,topic,heading_en,body_en,body_enscore,Classification
0,414,11-05-2017 00:39:13,ఐడిబిఐపై ఆర్బిఐ నజర్,భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ ...,business,RBI Nazar on IDBI,The Reserve Bank of India (RBI) has taken a cl...,0.017442,1
1,2668,12-06-2017 00:40:32,బ్యాంకింగ్ చీఫ్లతో నేడు జైట్లీ భేటీ,న్యూఢిల్లీ : ఆర్థిక మంత్రి అరుణ్ జైట్లీ సోమవార...,business,Jaitley met banking chiefs today,New Delhi: Finance Minister Arun Jaitley will ...,0.000000,1
2,19923,19-01-2017 19:51:31,కీలక వికెట్ తీసిన జడేజా..,కటక్: ఇంగ్లండ్తో జరుగుతున్న సెకండ్ వన్డే మ్యాచ...,sports,Jadeja took the key wicket..,Cuttack: In the second ODI match against Engla...,0.019608,1
3,15762,23-11-2017 17:29:04,మరో రెచ్చగొట్టే చర్యకు దిగిన పాకిస్థాన్,ఇస్లామాబాద్ : పాకిస్థాన్ అంతర్జాతీయ ఉగ్రవాది హ...,nation,Pakistan has taken another provocative action,Islamabad: Pakistan released international ter...,-0.030928,0
4,8624,03-04-2017 15:48:23,గోవాలో కొడుకుతో కలిసి అల్లు అర్జున్ స్విమ్మింగ్!,స్టార్ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటిక...,entertainment,Allu Arjun swimming with his son in Goa!,Despite being busy with a series of films as a...,0.078125,1


In [4]:
X = dataframe['body']

In [5]:
X.head()

0    భారీ ఎత్తున మొండిబకాయిలు పెరిగిపోవడంతో ఐడిబిఐ ...
1    న్యూఢిల్లీ : ఆర్థిక మంత్రి అరుణ్ జైట్లీ సోమవార...
2    కటక్: ఇంగ్లండ్తో జరుగుతున్న సెకండ్ వన్డే మ్యాచ...
3    ఇస్లామాబాద్ : పాకిస్థాన్ అంతర్జాతీయ ఉగ్రవాది హ...
4    స్టార్ హీరోగా వరుస సినిమాలతో బిజీగా ఉన్నప్పటిక...
Name: body, dtype: object

In [6]:
Y = dataframe["body_enscore"]

In [7]:
Y.head()

0    0.017442
1    0.000000
2    0.019608
3   -0.030928
4    0.078125
Name: body_enscore, dtype: float64

In [8]:
#df["topic"].unique()

In [9]:
# df["topic"] = df["topic"].replace("business", 1)
# df["topic"] = df["topic"].replace("sports", 2)
# df["topic"] = df["topic"].replace("nation", 3)
# df["topic"] = df["topic"].replace("entertainment", 4)
# df["topic"] = df["topic"].replace("editorial", 5)

In [10]:
#df["topic"].unique()

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 10)
X_train.head(4)

13674    85% కంప్యూటర్ నిరక్షరాస్యులుకొత్త తరంతో మాటామం...
3824     సైబర్ భద్రతపైనా దృష్టి పెట్టాలిన్యూఢిల్లీ : జి...
9037     టీమిండియాను ప్రకటించని బీసీసీఐ జట్ల ప్రకటనకు మ...
16872    2001లో కేసీఆర్ తెలంగాణ రాష్ట్ర సమితి పార్టీని ...
Name: body, dtype: object

In [12]:
y_train.head(4)

13674   -0.029787
3824    -0.007160
9037     0.006289
16872    0.000000
Name: body_enscore, dtype: float64

In [13]:
#pip install "tensorflow>=2.0.0"

In [14]:
#pip install --upgrade tensorflow-hub

In [15]:
#pip install tensorflow-text

In [16]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [17]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [18]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=1, batch_size = 32)

474/474 [==============================] - 2025s 4s/step - loss: 0.0481 - accuracy: 0.2192


In [19]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted
print(y_predicted)

203/203 [==============================] - 907s 4s/step
[[0.00685377]
 [0.00388049]
 [0.00655799]
 ...
 [0.00520475]
 [0.00819031]
 [0.00703144]]


In [20]:
print(y_test.head())

21597    0.000000
9561     0.000000
441     -0.023585
13317   -0.051724
7852    -0.025974
Name: body_enscore, dtype: float64


In [21]:
print(y_predicted)

[[0.00685377]
 [0.00388049]
 [0.00655799]
 ...
 [0.00520475]
 [0.00819031]
 [0.00703144]]


In [22]:
for each in y_predicted:
    print(each)

[0.00685377]
[0.00388049]
[0.00655799]
[0.00893885]
[0.00471397]
[0.00646128]
[0.00539082]
[0.0075222]
[0.00770864]
[0.00672029]
[0.0052098]
[0.00706858]
[0.00600795]
[0.00872209]
[0.00550308]
[0.00643529]
[0.00426926]
[0.00675781]
[0.00634742]
[0.00650613]
[0.00563827]
[0.0073428]
[0.02424885]
[0.00643109]
[0.01288279]
[0.00665055]
[0.00678817]
[0.00860447]
[0.00433954]
[0.00536059]
[0.00541767]
[0.00456325]
[0.00753383]
[0.00672452]
[0.01680265]
[0.00460637]
[0.00563231]
[0.00597371]
[0.00997215]
[0.0060901]
[0.00621206]
[0.00587264]
[0.0044295]
[0.00677002]
[0.00575396]
[0.0112378]
[0.00460915]
[0.00605781]
[0.00512529]
[0.00593211]
[0.00557053]
[0.00639889]
[0.01088507]
[0.00867204]
[0.00678625]
[0.0063431]
[0.00600918]
[0.00456476]
[0.00549998]
[0.00674833]
[0.00538148]
[0.00623895]
[0.01370709]
[0.00552968]
[0.00531246]
[0.00597126]
[0.00556013]
[0.00474022]
[0.00528871]
[0.00525776]
[0.01844725]
[0.00495938]
[0.01234391]
[0.00696357]
[0.00623498]
[0.00532411]
[0.00501545]
[0.005

[0.00638019]
[0.00510212]
[0.0075153]
[0.02340128]
[0.00531735]
[0.00456354]
[0.00655838]
[0.00567251]
[0.00526013]
[0.00631851]
[0.00447909]
[0.00644329]
[0.00410944]
[0.00472273]
[0.00477891]
[0.00563967]
[0.00599751]
[0.00486379]
[0.00490537]
[0.00482089]
[0.00644189]
[0.00670635]
[0.01031151]
[0.00503945]
[0.01528578]
[0.00541339]
[0.00852857]
[0.00497525]
[0.0052618]
[0.00508282]
[0.00501359]
[0.00575316]
[0.00571932]
[0.00509659]
[0.00695395]
[0.00635145]
[0.00589336]
[0.00503203]
[0.00500585]
[0.00553117]
[0.0063733]
[0.00568311]
[0.01013919]
[0.00579535]
[0.005506]
[0.00585119]
[0.00525288]
[0.0081724]
[0.00812922]
[0.00486119]
[0.00610101]
[0.00505367]
[0.00622794]
[0.00629224]
[0.00566837]
[0.00481117]
[0.00514283]
[0.00607965]
[0.0066086]
[0.00720883]
[0.00462366]
[0.0052393]
[0.00467159]
[0.00507699]
[0.00436107]
[0.00605205]
[0.00673934]
[0.00676477]
[0.00771895]
[0.00724182]
[0.0050183]
[0.00639169]
[0.00674317]
[0.00634998]
[0.00591588]
[0.00600851]
[0.01099797]
[0.00647

[0.00737774]
[0.00495292]
[0.00569603]
[0.0044105]
[0.0126968]
[0.00610506]
[0.00775656]
[0.00664124]
[0.00523518]
[0.00710887]
[0.00667636]
[0.0073441]
[0.01809185]
[0.00752755]
[0.00564476]
[0.00704996]
[0.00658311]
[0.00644175]
[0.0063043]
[0.00550382]
[0.00693707]
[0.00391223]
[0.01011107]
[0.00525673]
[0.00720917]
[0.00607862]
[0.00573298]
[0.00963895]
[0.00538765]
[0.00676722]
[0.00583766]
[0.00581188]
[0.00394002]
[0.00579686]
[0.00556324]
[0.0066056]
[0.00559282]
[0.01132793]
[0.00463419]
[0.00783301]
[0.00773868]
[0.00575737]
[0.00768493]
[0.00475161]
[0.00602398]
[0.00684389]
[0.00484882]
[0.00568634]
[0.00449442]
[0.00592985]
[0.00542358]
[0.00657464]
[0.00610358]
[0.00533531]
[0.00463729]
[0.0070886]
[0.00640323]
[0.01650771]
[0.00727209]
[0.00533429]
[0.00600548]
[0.00768462]
[0.00800295]
[0.00690311]
[0.00829818]
[0.00619335]
[0.00498542]
[0.00744436]
[0.00582337]
[0.00690316]
[0.00615723]
[0.00615037]
[0.0079108]
[0.00552166]
[0.0049489]
[0.00692904]
[0.00649411]
[0.0097

[0.00582279]
[0.00576819]
[0.00811626]
[0.00594925]
[0.00541426]
[0.00832726]
[0.00619722]
[0.00707307]
[0.0071096]
[0.00584442]
[0.00617761]
[0.00661528]
[0.00732337]
[0.00702437]
[0.00765361]
[0.00397661]
[0.00681066]
[0.00581517]
[0.00582544]
[0.00829798]
[0.0068633]
[0.00883398]
[0.00502238]
[0.00701191]
[0.00482281]
[0.00561295]
[0.00494815]
[0.00659705]
[0.00473535]
[0.00446711]
[0.00639623]
[0.00698288]
[0.0044237]
[0.00726945]
[0.00756892]
[0.00612517]
[0.00544647]
[0.00714658]
[0.00399858]
[0.00564114]
[0.00543464]
[0.00516547]
[0.00553412]
[0.00870861]
[0.00670765]
[0.00544986]
[0.00913547]
[0.00594896]
[0.00470471]
[0.00590521]
[0.00984057]
[0.00519387]
[0.00628391]
[0.01230969]
[0.00786093]
[0.00632939]
[0.00671944]
[0.00828014]
[0.00685226]
[0.00892452]
[0.00738273]
[0.00765125]
[0.00703234]
[0.01326084]
[0.00762213]
[0.01163784]
[0.00675776]
[0.00667585]
[0.00682619]
[0.00550891]
[0.00528982]
[0.00839045]
[0.0060136]
[0.00889514]
[0.01650736]
[0.00459569]
[0.00651021]
[0.

[0.00450022]
[0.00520393]
[0.00521765]
[0.00799906]
[0.00576338]
[0.00840951]
[0.0064247]
[0.00695888]
[0.00614778]
[0.00508524]
[0.00524537]
[0.00541435]
[0.00647215]
[0.00620029]
[0.00796536]
[0.0058021]
[0.00427929]
[0.00462715]
[0.00601953]
[0.00915245]
[0.00607585]
[0.00507531]
[0.00554155]
[0.00482218]
[0.00526249]
[0.0048276]
[0.01065278]
[0.00484405]
[0.00524612]
[0.00993224]
[0.00686092]
[0.0063678]
[0.00580408]
[0.00769533]
[0.01404697]
[0.00553676]
[0.00644012]
[0.00540872]
[0.01090158]
[0.00709935]
[0.00509007]
[0.00716862]
[0.00641228]
[0.00661588]
[0.00628856]
[0.00513695]
[0.00553866]
[0.00566078]
[0.01035054]
[0.01003953]
[0.00597736]
[0.00808992]
[0.00722293]
[0.00524309]
[0.00519598]
[0.00837117]
[0.00630413]
[0.00610258]
[0.00811371]
[0.00512071]
[0.00614458]
[0.00654838]
[0.0051962]
[0.00577748]
[0.00693881]
[0.00622318]
[0.00529115]
[0.00724968]
[0.00515134]
[0.00583597]
[0.00623129]
[0.00521836]
[0.00588597]
[0.00493238]
[0.00498001]
[0.01078079]
[0.00682941]
[0.0

[0.00579793]
[0.00585113]
[0.00533297]
[0.00475013]
[0.01004291]
[0.00551291]
[0.00480581]
[0.00789696]
[0.00654451]
[0.0067941]
[0.00599583]
[0.0051291]
[0.00533597]
[0.00594967]
[0.00584939]
[0.00805829]
[0.01034139]
[0.00674387]
[0.00790473]
[0.00553049]
[0.00518903]
[0.00465791]
[0.0047493]
[0.00422175]
[0.00583225]
[0.00482104]
[0.00475548]
[0.00743946]
[0.0062112]
[0.00671952]
[0.00501447]
[0.00524638]
[0.00415922]
[0.00800734]
[0.0067119]
[0.00996143]
[0.00514252]
[0.00432028]
[0.00454956]
[0.00629586]
[0.00538569]
[0.00394613]
[0.00517037]
[0.00559077]
[0.00579936]
[0.00898899]
[0.00499433]
[0.00560043]
[0.00788249]
[0.0059301]
[0.00450896]
[0.00550689]
[0.00504499]
[0.00679694]
[0.00713507]
[0.00868696]
[0.00511038]
[0.00468814]
[0.00554311]
[0.00776616]
[0.00554029]
[0.00582604]
[0.00613468]
[0.00624346]
[0.00595604]
[0.00490035]
[0.00876878]
[0.00631]
[0.00691323]
[0.00727888]
[0.00598832]
[0.00651052]
[0.00608929]
[0.00787671]
[0.00511466]
[0.00712549]
[0.00534326]
[0.00407

In [38]:
for each in np.argmax(y_predicted, axis = 1):
    print(each)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [23]:
X = dataframe['body']
Y = dataframe["Classification"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 10)

In [24]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
m = tf.keras.Model(inputs=[text_input], outputs = [l])
m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
m.fit(X_train, y_train, epochs=1, batch_size = 32)

474/474 [==============================] - 2144s 5s/step - loss: 0.6241 - accuracy: 0.7030


In [39]:
m.evaluate(X_test, y_test)

203/203 [==============================] - 897s 4s/step - loss: 0.5907 - accuracy: 0.7138


[0.5906646847724915, 0.7138456702232361]

In [25]:
y_pred = m.predict(X_test)
print(y_pred)

203/203 [==============================] - 921s 5s/step
[[0.72173697]
 [0.764458  ]
 [0.66365933]
 ...
 [0.72067463]
 [0.7299618 ]
 [0.63066894]]


In [67]:
#print(y_pred.flatten())
op = np.where(y_pred.flatten() < 0.7, 0, 1)

In [68]:
for each in op:
    print(each)

1
1
0
1
0
1
0
0
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
0
0
0
1
1
1
1
1
0
0
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
0
0
1
0
1
1
1
0
1
0
1
0
1
1
1
0
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
0
0
1
0
0
0
0
1
1
1
1
1
1
1
1
1
0
1
1
0
1
0
1
0
1
1
1
0
1
0
0
1
1
1
0
1
0
0
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
0
1
0
1
1
1
0
1
0
0
1
1
1
0
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
0
0
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
0
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
0
1
1
0
0
1
0
1
0
1
1
0
0
1
0
1
1
0
1
1
0
1
1
0
0
1
1
1
0
1
0
1
1
1
1
0
1
0
1
1
0
1
1
1
1
1
1
0
1
1
1
1
0
1
0
0
0
1
1
1
1
1
0
0
0
0
1
1
1
1
0
1
0
1
1
1
1
1
1
0
1
1
1
0
0
1
1
1
1
1
0
1
1
1
0
1
1
1
1
0
1
1
1
1
0
1
1
1
0
0
1
1
1
1
0
1
0
1
1
1
0
1
1
0
0
1
1
0
1
1
0
1
1
1
1
0
1
1
0
1
1
0
1
0
1
0
0
1
1
0
1
1
1
1
1
1
1
1
0
1
1
0
1
1
0
0
1
1
0
1
1
1
1
0
0
1
0
0
1
0
1
1
1
0
1
1
1
1
0
1
1
0
1
1
0
1
1
0
1
0
0
0
1
0
1
1
1
1
1
1
0
0
1
0
1
1
1
0
0
1
0
1
0
1
1
1
1
1
1
0
1
1
1
0
1
0
0
1
1
0
1
1
1
1
1
0
1
0
1
1
1
1
1


1
1
1
0
1
1
0
1
0
1
1
0
0
1
1
0
0
0
0
0
0
1
1
0
0
1
1
1
0
0
1
0
0
1
0
1
1
1
1
0
0
1
1
1
1
1
1
1
0
1
1
1
0
1
0
1
1
1
0
0
1
1
0
1
1
1
0
0
1
1
1
1
0
1
1
1
0
0
1
0
1
1
1
1
0
0
1
0
1
1
1
1
1
0
1
0
0
1
1
1
1
1
0
1
1
1
1
1
0
1
0
1
1
1
0
1
0
1
1
1
0
1
1
0
1
1
1
1
0
1
1
0
1
0
0
1
1
1
1
1
0
1
1
1
1
1
0
1
0
0
1
1
1
1
1
1
1
1
1
1
0
0
1
0
1
1
0
1
0
1
1
1
0
0
1
0
0
0
0
0
0
1
0
0
1
1
0
0
0
1
1
1
1
0
0
1
0
0
1
1
1
0
1
0
1
1
1
0
0
1
0
0
0
0
0
1
1
1
0
0
0
0
1
1
1
1
1
0
0
0
0
0
0
1
1
0
1
1
1
0
0
0
1
0
1
0
0
1
1
1
0
1
1
0
1
1
1
1
0
0
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
0
1
1
0
0
1
0
0
1
0
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
0
0
0
1
0
0
0
1
1
0
0
1
0
1
1
1
0
0
0
1
0
1
1
1
1
1
0
1
1
0
0
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
1
1
0
1
1
0
0
1
1
0
0
1
0
1
1
0
1
1
1
1
0
1
0
0
1
1
1
1
1
1
1
1
0
0
1
0
0
1
1
1
1
0
1
0
1
1
1
0
1
1
0
1
1
1
0
0
1
0
1
0
1
0
0
1
1
0
1
1
1
0
1
0
1
1
0
0
1
1
1
1
1
0
1
1
0
1
0
1
1
0
1
0
1
1
0
0
1
1
1
1
1
0
0
0
0
1
1
0
0
0
1
1
0
0
1
1
0
1
1
0
1
1
0
1
1
0
1
0
1
1
1
1
1
0
1
0


In [46]:
for each in y_pred:
    if(abs(each - 1) < abs(each-0.5)):
        print(1)
    else:
        print(0)

0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
1
0
0
1
0
0
0
0
1
0
1
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
1
1
1
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
1
1
1
0
0
0
1
1
1
1
0
1
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
1
1
0
0
1
0
0
0
0
0
1
1
0
1
0
1
0
1
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
1
0
1
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
1
0
0
1
0
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
1
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
1
0
1
1
0
0
0
0
0
1
0
1
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
1
1
0
0
1
0
1
0
1
0
1
0
0
0
0
0
1
1
1
0
0
1
1
1
0
0
0
0
0
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
1
1
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
1
0
1
0
0
1
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
1
1
0
1
1
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
1
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
1
0
0
0
0
1
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
0
1
0
0
0
1
0
0
0
1
0
0
1
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0
1
0
0
0
1
0
0
1
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
1
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
0
1
0
0
1
0
0
0
0
0
0
0
1
0
0
0
0
0
0
1
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
1
0


In [47]:
for each in y_test:
    print(each)

1
1
0
0
0
1
1
1
0
0
1
1
1
1
1
1
0
1
1
1
0
1
1
1
0
1
1
1
1
0
0
1
1
1
1
1
0
0
1
1
0
1
1
0
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
0
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
0
1
1
1
1
0
0
0
0
1
0
1
1
1
0
1
1
0
1
1
1
0
0
1
0
0
1
0
1
1
1
0
0
1
1
0
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
0
0
0
0
1
1
0
1
1
1
1
0
1
1
1
1
1
0
0
0
1
1
1
1
0
0
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
0
1
1
0
1
0
1
0
0
0
1
0
1
1
1
0
1
0
1
1
1
1
0
1
1
1
0
1
1
0
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
0
0
0
1
0
1
0
1
0
1
0
1
0
1
0
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
0
0
1
1
1
1
0
1
1
1
1
0
1
0
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
0
0
1
1
1
0
1
1
1
1
0
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
0
1
1
1
1
0
1
1
1
0
1
1
1
0
0
1
1
1
1
1
0
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
0


1
0
1
0
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
0
1
1
1
0
1
0
1
0
1
1
1
1
1
1
1
1
0
1
1
0
1
1
0
1
0
1
1
1
0
1
0
0
0
0
1
0
0
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
0
1
1
0
1
1
1
1
0
1
1
0
0
1
0
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
0
0
0
1
1
1
0
0
1
1
1
1
1
0
0
1
1
1
0
1
0
1
0
1
1
0
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
0
0
0
1
1
1
1
0
1
1
0
1
0
0
1
0
1
1
1
0
1
1
1
1
0
1
0
0
0
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
0
0
1
1
0
1
1
1
0
1
1
1
1
0
1
1
0
1
0
1
1
1
0
1
1
0
1
0
1
1
0
0
1
1
1
0
1
0
0
1
1
1
0
1
1
1
1
1
0
0
1
1
1
1
1
0
0
0
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
0
1
1
0
1
1
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
0
1
0
1
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
0
1
1
1
1
0
1
0
0
1
0
1
1
1
0
0
1
1
1
1
0
1
1
1
1
1
0
0
1
1
0
1
1
1
1
0
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
0
1
1
1
1
1
1
1
1
1
1
0
0
0
1
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
0
0
0
0
1
1
1
1
1
1
0
0
0
1
0


In [54]:
op = np.round(y_pred.flatten(),1).astype(np.int32)

In [55]:
for each in op:
    print(each)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [26]:
print(y_test.head())

21597    1
9561     1
441      0
13317    0
7852     0
Name: Classification, dtype: int64


In [28]:
print(y_pred)

[[0.72173697]
 [0.764458  ]
 [0.66365933]
 ...
 [0.72067463]
 [0.7299618 ]
 [0.63066894]]


In [37]:
for each in np.argmax(y_pred, axis = 1):
    print(each)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [31]:
X_test.head()

21597    పంజాబ్ చేతిలో కోహ్లీసేన చిత్తు139 ఛేదనలో 119కే...
9561     ఈ ఏడాదికి వెనక్కు తగ్గినకేంద్రంనీట్పై అధ్యయనం ...
441      న్యూఢిల్లీ, మే 6: దేశ రాజధాని ఢిల్లీలో ఓ కంటెయ...
13317    రాష్ట్రవ్యాప్తంగా విద్యుత్ ట్రాన్స్ఫార్మర్లు, ...
7852     న్యూఢిల్లీ: జిఎస్టీలో పన్ను ఎగవేతలకు ఆస్కా రం ...
Name: body, dtype: object

In [33]:
y_test.head()

21597    1
9561     1
441      0
13317    0
7852     0
Name: Classification, dtype: int64